## 모듈 임포트

In [1]:
import requests
from bs4 import BeautifulSoup

import time
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium import webdriver

from selenium.webdriver import ActionChains

import pandas as pd
import numpy as np

from geopy.geocoders import Nominatim
from tqdm import tqdm

import folium

import os
FILE_PATH = "DATA/naver_daegu_jungu_restaurant.xlsx"

### 네이버 지도에 "대구 중구 식당"이라는 키워드 검색 결과 나오는 식당들 셀레니움으로 주소만 추출

In [2]:
if os.path.exists(FILE_PATH):
    print("파일이 이미 존재합니다.")
else:
    url = 'https://map.naver.com/p/search/대구%20중구%20식당'
    
    options = Options()
    options.add_experimental_option("detach", True)
    
    driver = webdriver.Chrome(options=options)
    driver.implicitly_wait(3)
    
    driver.get(url)
    
    # 프레임 이동
    driver.switch_to.frame('searchIframe')
    
    # 끝까지 스크롤
    while True:
        last_li = driver.find_elements(By.CSS_SELECTOR, 'li.UEzoS')[-1]
        
        action = ActionChains(driver)
        action.move_to_element(last_li).perform()
    
        new_last_li = driver.find_elements(By.CSS_SELECTOR, 'li.UEzoS')[-1]
    
        if last_li == new_last_li:
            break
    
    row = []
    
    while True:
        
        rest_num = len(driver.find_elements(By.CSS_SELECTOR, 'span.place_bluelink.TYaxT'))
        
        for i in range(rest_num):
            try:
                rest_element = driver.find_elements(By.CSS_SELECTOR, 'span.place_bluelink.TYaxT')[i]
                rest_title = rest_element.text
                rest_element.click()
                time.sleep(3)
    
                # 프레임 이동
                driver.switch_to.default_content()
                driver.switch_to.frame('entryIframe')
                
                rest_loc = driver.find_element(By.CSS_SELECTOR, 'span.LDgIH').text # 식당 주소
                row.append([rest_title, rest_loc])
    
                # 프레임 이동
                driver.switch_to.default_content()
                driver.switch_to.frame('searchIframe')
                
            except Exception as e:
                print(f"Error occurred for element {i}: {str(e)}") # 오류 발생시 continue
                continue
                
        next_button = driver.find_elements(By.CSS_SELECTOR, 'a.eUTV2')[1] # 다음 버튼
        
        if next_button.get_attribute('aria-disabled') == 'true': # 다음 버튼 속성에 '클릭 불가'를 의미하는 true이면 반복문 out
            break
            
        next_button.click()
        time.sleep(3)
    
    driver.quit()

    rest_add_df = pd.DataFrame(row, columns=['title', 'address'])
    rest_add_df.set_index('title', inplace=True)

파일이 이미 존재합니다.


In [3]:
# rest_add_df = pd.DataFrame(row, columns=['title', 'address'])
# rest_add_df.set_index('title', inplace=True)
# rest_add_df.head()

### iframe에 src 속성에 있는 url 이용해서 soup로 나머지 정보 추출
- 위 코드와 거의 동일

In [4]:
if os.path.exists(FILE_PATH):
    print("파일이 이미 존재합니다.")
else:
    url = 'https://map.naver.com/p/search/대구%20중구%20식당'
    
    options = Options()
    options.add_experimental_option("detach", True)
    
    driver = webdriver.Chrome(options=options)
    driver.implicitly_wait(3)
    
    driver.get(url)
    
    detail_page_url = driver.find_element(By.CSS_SELECTOR, '#searchIframe').get_attribute('src')
    print(detail_page_url)
    
    driver.quit()

파일이 이미 존재합니다.


In [5]:
if os.path.exists(FILE_PATH):
    print("파일이 이미 존재합니다.")
else:
    options = Options()
    options.add_experimental_option("detach", True)
    
    driver = webdriver.Chrome(options=options)
    
    driver.implicitly_wait(3)
    
    driver.get(detail_page_url)
    time.sleep(2)
    
    while True:
        last_li = driver.find_elements(By.CSS_SELECTOR, 'li.UEzoS')[-1]
        
        action = ActionChains(driver)
        action.move_to_element(last_li).perform()
    
        new_last_li = driver.find_elements(By.CSS_SELECTOR, 'li.UEzoS')[-1]
    
        if last_li == new_last_li:
            break
    
    pages = []
    pages.append(driver.page_source)
    
    while True:
        next_button = driver.find_elements(By.CSS_SELECTOR, 'a.eUTV2')[1]
        
        if next_button.get_attribute('aria-disabled') == 'true':
            break
            
        next_button.click()
        
        time.sleep(3)
    
        pages.append(driver.page_source)
        time.sleep(2)
    
    driver.quit()

파일이 이미 존재합니다.


In [6]:
if os.path.exists(FILE_PATH):
    print("파일이 이미 존재합니다.")
else:
    row = []
    for i, page in enumerate(pages):
        soup = BeautifulSoup(page, 'html.parser')
        li_tags = soup.select('li.UEzoS')
    
        print(f"{i+1} page loading success")
        
        for j, li_tag in enumerate(li_tags):
            rest_name = li_tag.select_one('span.place_bluelink.TYaxT').get_text() # 식당 이름
            rest_cat = li_tag.select_one('span.KCMnt').get_text() # 식당 카테고리
            
            if li_tag.select_one('span.h69bs.orXYY') is None: # 별점이 있냐 없냐에 따라 리뷰 개수 태그를 추출하는 인덱스가 달라짐
                rest_star = np.nan
                rest_rev_count = li_tag.select('span.h69bs')[1].get_text()
    
                if li_tag.select_one('span.h69bs.DjPAB') is not None:
                    rest_star = np.nan
                    rest_rev_count = li_tag.select('span.h69bs')[2].get_text()
            else:
                rest_star = li_tag.select_one('span.h69bs.orXYY').get_text() # 별점
                rest_rev_count = li_tag.select('span.h69bs')[2].get_text() # 리뷰 개수
    
            if li_tag.select_one('div.u4vcQ') is None:
                rest_rev = np.nan
            else:
                rest_rev = li_tag.select_one('div.u4vcQ').get_text() # 상단 리뷰
            
            row.append([rest_name, rest_cat, rest_star, rest_rev_count, rest_rev])

파일이 이미 존재합니다.


In [7]:
if os.path.exists(FILE_PATH):
    print("파일이 이미 존재합니다.")
else:
    data_df = pd.DataFrame(row, columns=['title', 'category', 'star', 'review_count', 'review'])
    data_df.set_index('title', inplace=True)
    data_df.head()

파일이 이미 존재합니다.


In [8]:
if os.path.exists(FILE_PATH):
    print("파일이 이미 존재합니다.")
else:
    joined_df = data_df.join(rest_add_df)
    joined_df.head()

파일이 이미 존재합니다.


In [9]:
if os.path.exists(FILE_PATH):
    print("파일이 이미 존재합니다.")
else:
    joined_df.reset_index().to_excel("DATA/naver_daegu_jungu_restaurant.xlsx", index=False)

파일이 이미 존재합니다.


In [10]:
df = pd.read_excel("DATA/naver_daegu_jungu_restaurant.xlsx")

In [11]:
df.shape

(314, 6)

In [12]:
df[df['address'].isnull()]

,title,category,star,review_count,review,address
4,904,"맥주,호프",NaN,리뷰 78,NaN,NaN
225,와래이수제꼬치전문점,"일식튀김,꼬치",별점4.88,리뷰 535,NaN,NaN
261,주담,술집,NaN,리뷰 57,NaN,NaN


## 주소 채우고 지도에 가게 표시

In [13]:
df['simple_address'] = df['address'].str.split(' ').str[:4].apply(lambda x: ' '.join(x) if isinstance(x, list) else '')

# 위도, 경도 반환하는 함수
def geocoding(address):
    try:
        geo_local = Nominatim(user_agent='South Korea')  #지역설정
        location = geo_local.geocode(address)
        geo = [location.latitude, location.longitude]
        return geo

    except:
        return [0,0]

# 실행
for idx,addr in enumerate(tqdm(df.simple_address)):
    df.loc[idx,'latitude'] = geocoding(addr)[0]
    df.loc[idx,'longitude'] = geocoding(addr)[1]

100%|████████████████████████████████████████████████████████████████████████████████| 314/314 [11:00<00:00,  2.10s/it]


In [14]:
# 식당이름 중복 삭제
df1 = df[~df.duplicated(subset='title', keep=False)].reset_index().drop('index', axis=1)
df2 = df[df.duplicated(subset='title', keep=False) & df['review'].notna()]
df = pd.concat([df1, df2], ignore_index=True)

In [15]:
df.head()

,title,category,star,review_count,review,address,simple_address,latitude,longitude
0,666육육육 동성로점,요리주점,NaN,리뷰 431,마라샹궈랑 카이막플레이트 너무 맛있었어요 ! 카이막은 술집에서 파는 카이막 중에서 ...,대구 중구 동성로4길 28-12 1층 666,대구 중구 동성로4길 28-12,35.864723,128.599288
1,8,와인,NaN,리뷰 276,"입구부터 특이하면서 독특한 분위기였는데, 음식도 깔끔하고 와인, 칵테일이랑도 굉장히...",대구 중구 동성로1길 29-17 1층,대구 중구 동성로1길 29-17,35.867264,128.594891
2,86계성집 팔륙계성집,닭볶음탕,별점4.87,리뷰 875,닭구이 단품 주문하면 라면이랑 비빔밥까지 나와요,대구 중구 중앙대로81길 82 팔륙계성집,대구 중구 중앙대로81길 82,35.869772,128.593734
3,8번식당 본점,"순대,순댓국",별점4.38,리뷰 999+,"수육이랑 순대가 잡내1도 없이 찐이고요, 깍두기도 존맛이에요. ㅋㅋ 대구여행으로 오...",대구 중구 서성로13길 8 1층,대구 중구 서성로13길 8,35.872062,128.585264
4,904,"맥주,호프",NaN,리뷰 78,NaN,NaN,,0.000000,0.000000


In [16]:
color_dict = {
    '동구': '#00C7FF',
    '서구': '#EE0613',
    '남구': 'orange',
    '북구': 'green',
    '중구': 'violet',
    '수성구': 'brown',
    '달성군': 'cyan',
    '달서구': 'magenta'
}

In [17]:
lat = df['latitude'].median()
long = df['longitude'].median()

m = folium.Map(location=[lat,long], zoom_start=14, height=800)

for i in df.index:
    row = df.loc[i]
    i_lat = row['latitude']
    i_long = row['longitude']
    label = row['title']+' - '+row['simple_address'][3:] # 도로명 주소앞 '대구광역시 ' 생략
    # color = color_dict[row['브랜드명']]
    popup = folium.Popup(label, max_width=150)
    folium.CircleMarker(
        location = [i_lat, i_long],
        radius = 3,
        # color = color,
        fill = False,
        fill_opacity = 1, 
        # fill_color = color,
        popup = popup
    ).add_to(m)
m